# Matched Filter for Phase Measurement

Matched filters are typically used for digital phase / signal delay measurement, however, this is usually done when the measured delay is large (typically >>10 ns). To measure a delay amounting to 1 cm, the method needs to be able to measure 30 ps time difference since speed of light is 300M m/s. This requires sampling at prohibitively large rates (>10 GHz !!), thus, it's impractical.

Still we show how it works below

In [1]:
import numpy             as np
import matplotlib.pyplot as plt
import time
from scipy           import signal

# bad matched filter (approximate)

https://stackoverflow.com/questions/19468733/how-to-implement-a-matched-filter

In [2]:
f_e = 1e6;
dt = 1e-10;
stop = 1e-4;
delay = 5e-8;
stdev = 0.001;
atten = 0.01;

s_simulation = np.linspace(dt, stop, int(stop/dt));
s_e  = np.sin(2*np.pi* f_e *s_simulation - np.pi/32);
s_rc = np.sin(2*np.pi* f_e *(s_simulation - delay) - np.pi/32)
s_r  = atten*np.sin(2*np.pi* f_e *(s_simulation - delay) - np.pi/32) + stdev*np.random.randn((len(s_simulation)));
atten_est = np.sqrt(np.mean(s_r*s_r))*np.sqrt(2)
print(atten_est)
template = s_e[0:int(1/(f_e*dt))];
b = np.flipud(template)
x = s_r
n = range(0,len(x))
y = signal.lfilter(b, np.ones(1), x);
thresh = 0.9
u = atten_est*np.dot(template, template);
aa = y>(thresh*u)
matches = np.asarray(n)[aa];
steps = np.concatenate((np.asarray([0]), np.where(np.gradient(matches)>1)[0][1::2]))
means = np.zeros((steps.shape[0]-1));
for i in range(1,steps.shape[0]):
    means[i-1] = np.mod( np.mean(matches[steps[i-1]:steps[i]]), int(1/(f_e*dt)) );
print("delay_est:", np.mean(means)*dt, ". actual delay:", delay)

0.010099996698932096
delay_est: 4.989848484848485e-08 . actual delay: 5e-08


# true matched filter

In [3]:
f_e = 1e6;
dt = 1e-10;
stop = 0.011;
delay = 4.3e-8;
stdev = 0.00001;
atten = 0.01;

s_simulation = np.linspace(dt, stop, int(stop/dt));
s_e  = np.sin(2*np.pi* f_e *s_simulation - np.pi/32);
s_r  = atten*np.sin(2*np.pi* f_e *(s_simulation - delay) - np.pi/32) + stdev*np.random.randn((len(s_simulation)));

In [4]:
y = signal.correlate(s_e, s_r, 'same');

In [5]:
print("measured delay:", ((stop/(dt*2))-np.argmax(y))*dt)

measured delay: 4.3099999999254944e-08
